In [4]:
pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 6.2 MB 2.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 281 kB 63.6 MB/s 
     |████████████████████████████████| 36.8 MB 1.2 MB/s 
     |████████████████████████████████| 51 kB 742 kB/s 
     |████████████████████████████████| 103 kB 57.2 MB/s 
     |████████████████████████████████| 78 kB 6.6 MB/s 
     |████████████████████████████████| 109 kB 73.3 MB/s 
     |████████████████████████████████| 45 kB 4.1 MB/s 
     |████████████████████████████████| 41 kB 442 kB/s 
     |████████████████████████████████| 3.2 MB 43.3 MB/s 
     |████████████████████████████████| 546 kB 48.7 MB/s 
     |████████████████████████████████| 100 kB 11.9 MB/s 
     |████████████████████████████████| 121 kB 63.0 MB/s 
     |████████████████████████████████| 90 kB

In [1]:
args = {'data':'cora',
        'h_feats':16,
        'lr':0.01,
        'epochs':100}

In [6]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--data', type=str, default = "cora", help = "benchmark dataset")
# parser.add_argument('--h_feats', type=int, default = 16, help = "number of hidden channels")
# parser.add_argument('--lr', type=float, default = 0.01, help = "learning rate")
# parser.add_argument('--epochs', type=int, default = 100, help = "epochs")
# args = parser.parse_args()

data = str(args['data'])
h_feats = int(args['h_feats'])
lr = float(args['lr'])
epochs = int(args['epochs'])

if data == "cora":
    dataset = dgl.data.CoraGraphDataset()
elif data == "citeseer":
    dataset = dgl.data.CiteseerGraphDataset()
elif data == "pubmed":
    dataset = dgl.data.PubmedGraphDataset()

print('Number of categories:', dataset.num_classes)

g = dataset[0]


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(epochs):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % (epochs/20) == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

model = GCN(g.ndata['feat'].shape[1], h_feats, dataset.num_classes)
train(g, model)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 1.946, val acc: 0.170 (best 0.170), test acc: 0.182 (best 0.182)
In epoch 5, loss: 1.893, val acc: 0.370 (best 0.372), test acc: 0.430 (best 0.424)
In epoch 10, loss: 1.813, val acc: 0.416 (best 0.416), test acc: 0.445 (best 0.445)
In epoch 15, loss: 1.711, val acc: 0.482 (best 0.482), test acc: 0.504 (best 0.504)
In epoch 20, loss: 1.586, val acc: 0.542 (best 0.542), test acc: 0.577 (best 0.577)
In epoch 25, loss: 1.443, val acc: 0.604 (best 0.604), test acc: 0.639 (best 0.639)
In epoch 30, loss: 1.284, val acc: 0.650 (best 0.650), test acc: 0.671 (best 0.671)
In epoch 35, loss: 1.118, val acc: 0.684 (best 0.684), test acc: 0.698 (best 0.698)
In epoch 40, loss: 0.951, val acc: 0.702 (best 0.704), test acc: 0.715 (best 0.714)
In epoch 45, loss: 0.792, val acc: 0.726 (best 0.726), test acc: 0.724 (best 0.722)
In epoch 50, loss: 0.649, val acc: 0.746 (best 0.746), test acc: 0.731 (best 0.728)
In epoch 55, loss: 0.525, val acc: 0.760 (best 0.760), test acc: 0.735 (best 0